In [1]:
import kagglehub
from transformers import Idefics2Processor, Idefics2ForConditionalGeneration, BitsAndBytesConfig

import cv2
import torch
from PIL import Image
import pandas as pd
import os
os.environ["YOLO_VERBOSE"] = "False"  # Prevent auto-downloads
from ultralytics import YOLO
from tqdm import tqdm
import json
iou_threshold = 0.5

/home/s3544648/.conda/envs/dev/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
path_private = "./data"
path_private


'./data'

In [3]:
path_public = "/home/s3544648/data/public-set"
path_public

'/home/s3544648/data/public-set'

In [4]:
df_private = pd.read_csv( os.path.join(path_private, "labels_dataframe.csv"))
df_private.head()

,Task ID,Task Name,Job Id,Source,Frames,Absolute Frame,Relative Frame,XTL,YTL,XBR,YBR,Code,Issue
0,138,Task1,133,1690279852.mp4,730,54,54,29.87,506.88,190.69,554.96,83/2789,NaN
1,138,Task1,133,1690279852.mp4,730,55,55,65.26,504.87,225.50,552.95,83/2789,NaN
2,138,Task1,133,1690279852.mp4,730,56,56,131.98,503.67,291.63,551.76,83/2789,NaN
3,138,Task1,133,1690279852.mp4,730,57,57,198.69,502.48,357.76,550.57,83/2789,NaN
4,138,Task1,133,1690279852.mp4,730,58,58,241.62,498.68,400.10,546.77,83/2789,NaN


In [5]:
df_public = pd.read_csv(os.path.join(path_public, "images_with_boxes.csv"))
df_public.head()

,image_id,image_name,task_id,width,height,box_label,box_source,box_xtl,box_ytl,box_xbr,box_ybr,issue,code
0,10,images/10.png,226,1000,668,hazmat_sign,manual,825.87,413.29,855.01,437.19,low quality,90/3082
1,100,images/100.png,226,809,1080,hazmat_sign,manual,187.30,571.99,248.56,618.26,NaN,22/1965
2,101,images/101.png,226,812,1080,hazmat_sign,manual,541.74,507.06,654.83,606.01,NaN,23/1049
3,102,images/102.png,226,810,1079,hazmat_sign,manual,426.69,474.05,566.98,576.68,NaN,23/1049
4,103,images/103.png,226,808,1080,hazmat_sign,manual,320.08,722.87,451.08,840.67,NaN,30/1202


In [6]:
df_private.groupby("Absolute Frame").count()

,Task ID,Task Name,Job Id,Source,Frames,Relative Frame,XTL,YTL,XBR,YBR,Code,Issue
Absolute Frame,,,,,,,,,,,,
54,1,1,1,1,1,1,1,1,1,1,1,0
55,1,1,1,1,1,1,1,1,1,1,1,0
56,1,1,1,1,1,1,1,1,1,1,1,0
57,1,1,1,1,1,1,1,1,1,1,1,0
58,1,1,1,1,1,1,1,1,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
59835,1,1,1,1,1,1,1,1,1,1,1,1
59836,1,1,1,1,1,1,1,1,1,1,1,1
59837,1,1,1,1,1,1,1,1,1,1,1,1


In [7]:
filtered_df = df_private.groupby("Absolute Frame").filter(
    lambda group: group["Code"].nunique() > 1
)

# Display the filtered DataFrame
filtered_df.groupby("Absolute Frame").count()

,Task ID,Task Name,Job Id,Source,Frames,Relative Frame,XTL,YTL,XBR,YBR,Code,Issue
Absolute Frame,,,,,,,,,,,,
36647,2,2,2,2,2,2,2,2,2,2,2,1
36648,2,2,2,2,2,2,2,2,2,2,2,1
36651,2,2,2,2,2,2,2,2,2,2,2,1
36652,2,2,2,2,2,2,2,2,2,2,2,1
36676,2,2,2,2,2,2,2,2,2,2,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...
36772,2,2,2,2,2,2,2,2,2,2,2,1
36773,2,2,2,2,2,2,2,2,2,2,2,1
36774,2,2,2,2,2,2,2,2,2,2,2,1


In [8]:
model_detection =  YOLO('/home/s3544648/myjupyter/un-numbers/UN-number-detection/runs/detect/train_31epoch/weights/best.pt')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_detection.to(device)


YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 96, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(96, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(96, 192, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(192, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C3k2(
        (cv1): Conv(
          (conv): Conv2d(192, 192, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(192, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(384, 384, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(384, eps=0.001, momentum=0.03, affine=True, track_

In [9]:
processor = Idefics2Processor.from_pretrained("./idefics2")
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16,
)
print('Processor loaded')
print('Device:', device)
model_ocr = Idefics2ForConditionalGeneration.from_pretrained(
    "./idefics2",
    torch_dtype=torch.float16,
    device_map=device,
    quantization_config=quantization_config,   
    # attn_implementation="flash_attention_2",
)
model_ocr = model_ocr.to(device)


Chat templates should be in a 'chat_template.jinja' file but found key='chat_template' in the processor's config. Make sure to move your template to its own file.


Processor loaded
Device: cuda


Loading checkpoint shards: 100%|██████████| 7/7 [01:09<00:00,  9.95s/it]


In [10]:
prompt = """
Analyze the image and extract two key values:

    The UN number visible on the upper part of the placard.
    The code visible on the lower part of the placard, located below the horizontal line separating the two sections.

Both codes are printed in black. If either the upper or lower part cannot be detected, replace the missing value with "0." Output the extracted values as plain text, separated by a comma if multiple codes are present. No additional context or formatting is needed.

Input Examples:

    {98 {line} 4567}
    (not found, {line}, 8901)
    {101 {line} 3345}
    (not found, {line}, {not found})
    {45 {line} 2789}
    {22 {line} 5678}

Desired Output:

    98, 4567
    0, 8901
    101, 3345
    0, 0
    45, 2789
    22, 5678

Expected Transformation:

    For each input example, extract the UN number and the code below the horizontal line.
    If either part is missing (i.e., "not found"), replace it with 0.
    Output the extracted values as plain text, separated by a comma, without any additional context or formatting.
"""
messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "text",
                "text": prompt},
            {"type": "image"},
        ],
    }
]
text = processor.apply_chat_template(messages, add_generation_prompt=True)

In [ ]:

def get_un_number(image, bbox):
    pil_img = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    # crop image
    cropped_img = pil_img.crop(bbox)
    # OCR
    return perform_ocr(cropped_img)
def calculate_iou(box1, box2):
    """
    Calculate Intersection over Union (IoU) between two bounding boxes.
    box1 and box2 are in the format (x_min, y_min, x_max, y_max).
    """
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])

    # Compute the area of intersection
    intersection_area = max(0, x2 - x1) * max(0, y2 - y1)

    # Compute the area of both bounding boxes
    box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
    box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])

    # Compute the union area
    union_area = box1_area + box2_area - intersection_area

    # Avoid division by zero
    if union_area == 0:
        return 0

    # Compute IoU
    iou = intersection_area / union_area
    return iou   
def perform_ocr(image): 
    inputs = processor(images=image, text=text, return_tensors="pt").to(device)
    generated_text = model_ocr.generate(**inputs, max_new_tokens=500)
    generated_text = processor.batch_decode(generated_text, skip_special_tokens=True)[0]
    assistant_output = generated_text.split("Assistant:")[1].strip()
    
    # Split the output by comma to get the individual numbers
    numbers = assistant_output.split(",")

    numbers = [number.strip().replace('.','') for number in numbers]
    numbers.append('0000') 
    numbers.append('0000')
    un_number, hin_number = numbers[:2]
    return un_number, hin_number  

def get_bboxes(image_path):
    img = cv2.imread(image_path)
    results = model_detection(img)
    return results, img     
    
def get_ground_truth(image_path, bbox: tuple[float, float, float, float]):
    image_name = image_path.split("\\")[-1]
    predictions = None
    if '_' in image_name:
        video_name = image_name.split('_')[0]
        frame_id = int(image_name.split('_')[1].split('.')[0])
        predictions = df_private[(df_private['Source'] == video_name + '.mp4') & (df_private['Relative Frame'] == int(frame_id))]
        
    else:
        image_id = image_name.split('.')[0]
        predictions = df_public[df_public['image_id'] == int(image_id)]
    # Check if the predicted bbox matches any ground truth bbox
    if predictions is not None and not predictions.empty:
        # Check for the existence of 'XTL' or 'box_xtl' columns
        xtl_column = 'XTL' if 'XTL' in predictions.columns else 'box_xtl'
        ytl_column = 'YTL' if 'YTL' in predictions.columns else 'box_ytl'
        xbr_column = 'XBR' if 'XBR' in predictions.columns else 'box_xbr'
        ybr_column = 'YBR' if 'YBR' in predictions.columns else 'box_ybr'

        # Iterate through predictions to check IoU
        for _, row in predictions.iterrows():
            ground_truth_bbox = (row[xtl_column], row[ytl_column], row[xbr_column], row[ybr_column])
            iou = calculate_iou(bbox, ground_truth_bbox)
            if iou > iou_threshold:  # IoU threshold
                # Return the 'Code' column if it exists, otherwise return "code"
                code = row['Code'] if 'Code' in row else row["code"]
                return code.split('/')

    # If no valid prediction is found, return "code"
    return None
def store_result(res):
    print('Saving evaluation')
    with open('full-pipeline-yolo-evaluation.json', 'w') as f:
        json.dump(res, f)
    print('Evaluation saved')        
    

In [17]:
test_metrics = {"metrics": [], "success": 0, "failure": 0}

def test_path(path):
    # list all png jpeg files in the path
    files = [f for f in os.listdir(path) if f.endswith('.png') or f.endswith('.jpg')]
    with tqdm(total=len(files), desc="Processing images", unit="image") as pbar:
        for file in files:
            image_path = os.path.join(path, file)
            [results, img] = get_bboxes(image_path)
            if results is not None:
                for result in results:  # Assuming result is a list of bounding boxes
                    for bbox in result.boxes:
                        bbox = bbox.xyxy[0].tolist()
                        ground_truth = get_ground_truth(image_path, bbox)
                        un_number, hin_number = get_un_number(img, bbox)
                        if ground_truth is not None:
                            if ground_truth[1] == hin_number and ground_truth[0] == un_number:
                                test_metrics["success"] += 1
                            else:
                                test_metrics["failure"] += 1
                        else:
                            test_metrics["failure"] += 1
                        test_metrics["metrics"].append({
                            "image": image_path,
                            "bbox": bbox,
                            "ground_truth": ground_truth,
                            "prediction": [un_number, hin_number]
                        })
            # Update the progress bar and description
            pbar.set_description(f"Processing images | Success: {test_metrics['success']} | Failure: {test_metrics['failure']}")
            pbar.update(1)
        
        

In [13]:
path = os.path.join(path_public, "images")
print(path)
test_path(os.path.join("/home", "s3544648", "data", "yolo", "images", "test"))
test_path(os.path.join("/home", "s3544648", "data", "yolo", "images", "val"))
test_path(os.path.join("/home", "s3544648", "data", "yolo", "images", "train"))
test_path(os.path.join(path_public, "images"))
store_result

/home/s3544648/data/public-set/images


Processing images | Success: 0 | Failure: 18:   2%|▏         | 18/979 [01:42<1:31:38,  5.72s/image]


ValueError: not enough values to unpack (expected 2, got 1)

In [ ]:
print(test_metrics["metrics"])